<a href="https://colab.research.google.com/github/anastasia-evil/MyApplication/blob/master/diezmado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Primero, instala las bibliotecas necesarias si aún no lo has hecho.
!pip install pandas

import pandas as pd
import os
from google.colab import drive

# Montar Google Drive para acceder a los archivos
drive.mount('/content/drive')

# Ruta a la carpeta "TFG" en tu Google Drive
folder_path = '/content/drive/My Drive/TFGDatos/'
output_folder_path = '/content/drive/My Drive/TFGDatos_Diezmado/'

Mounted at /content/drive


In [ ]:
def reduce_resolution(df, time_col, label_col, target_freq=100):
    # Convertir la columna de tiempo a timedelta
    df[time_col] = pd.to_timedelta(df[time_col])

    # Obtener el intervalo de muestreo para la nueva frecuencia
    interval = pd.to_timedelta(1 / target_freq, unit='s')

    # Crear un nuevo DataFrame para almacenar los datos reducidos
    reduced_df = pd.DataFrame(columns=df.columns)

    current_time = df[time_col].iloc[0]
    end_time = df[time_col].iloc[-1]

    while current_time <= end_time:
        # Filtrar las filas en el intervalo actual
        mask = (df[time_col] >= current_time) & (df[time_col] < current_time + interval)
        interval_df = df[mask]

        if not interval_df.empty:
            # Mantener la fila con la etiqueta no cero si existe, de lo contrario la primera fila
            if interval_df[label_col].any():
                selected_row = interval_df[interval_df[label_col] != 0].iloc[0]
            else:
                selected_row = interval_df.iloc[0]

            # Usar concat en lugar de append
            reduced_df = pd.concat([reduced_df, selected_row.to_frame().T])

        current_time += interval

    return reduced_df


In [ ]:
# Obtener la lista de archivos en la carpeta de origen
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

for file in files:
    # Cargar el archivo Excel
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path)

    # Reducir la resolución del DataFrame
    reduced_df = reduce_resolution(df, 'Time', 'Label', target_freq=100)

    # Guardar el nuevo DataFrame en la carpeta de destino
    output_file_path = os.path.join(output_folder_path, file)
    reduced_df.to_excel(output_file_path, index=False)

In [ ]:
import pandas as pd
import os
import glob

# Función para verificar si la columna 'Label' tiene algún valor distinto de 0
def check_labels(file_path):
    df = pd.read_excel(file_path)
    if 'Label' in df.columns:
        if (df['Label'] != 0).any():
            return False
    return True

# Ruta a la carpeta que contiene los archivos Excel
path = '/content/drive/My Drive/TFGDatos_Diezmado'
xlsx_files = glob.glob(os.path.join(path, '*.xlsx'))

# Lista para almacenar los nombres de los archivos que no tienen valores distintos de 0 en 'Label'
files_without_labels = []

for file in xlsx_files:
    if check_labels(file):
        files_without_labels.append(file)

# Mostrar los archivos que no tienen valores distintos de 0 en la columna 'Label'
if files_without_labels:
    print("Archivos que no tienen valores distintos de 0 en la columna 'Label':")
    for file in files_without_labels:
        print(file)
else:
    print("Todos los archivos tienen al menos un valor distinto de 0 en la columna 'Label'.")


Archivos que no tienen valores distintos de 0 en la columna 'Label':
/content/drive/My Drive/TFGDatos_Diezmado/[p197] low-2 BITalino.xlsx
/content/drive/My Drive/TFGDatos_Diezmado/[p206] low-2 BITalino.xlsx
/content/drive/My Drive/TFGDatos_Diezmado/[p212] control-1 BITalino.xlsx
